# 示例应用：电影评论的情感分析

示例使用由斯坦福研究员Andrew Maas收集的IMDb (Internet Movie Database,互联网电影数据库) 网站的电影评论数据集。这个数据集包含评论文本，还有一个标签，用于表示该评论是"正面的”(positive)还是"负面的" (negative)。IMDb网站本身包含从1到 10的打分。为了简化建模，这些评论打分被归纳为一个二分类数据集，评分大于等于7 的评论被标记为"正面的"，评分小干等于4 的评论被标记为"负面的"，中性评论没有包含在数据集中。我们不讨论这种方法是否是一种好的数据表示，而只是使用Andrew Maas提供的数据。 

将数据解压之后，数据集包括两个独立文件夹中的文本文件，一个是训练数据， 一个是测试数据。每个文件夹又都有两个子文件夹，一个叫作pos, 一个叫作neg：

pos文件夹包含所有正面的评论，每条评论都是一个单独的文本文件，neg文件夹与之类似。scikit-learn中有一个辅助函数可以加载用这种文件夹结构保存的件，其中每个子文件夹对应一个标签，这个函数叫作load_files。我们首先将load_fties函数应用于训练数据：

In [25]:
from sklearn.datasets import load_files
import numpy as np
reviews_train = load_files("E:/clone/machine-learning/data/aclImdb/train/")
# load_files返回一个Bunch对象，其中包含训练文本和训练标签
'''labeled_indices = reviews_train.target != 2
text_train, y_train = reviews_train.data[labeled_indices], reviews_train.target[labeled_indices]'''

data_array = np.array(reviews_train.data)
target_array = np.array(reviews_train.target)

# Filter out documents where the target is not equal to 2
labeled_indices = np.where(target_array != 2)[0]
text_train, y_train = data_array[labeled_indices], target_array[labeled_indices]
print("type of text_train : {}" .format(type(text_train))) 
print("length of text_train : {}".format(len(text_train))) 
print("text_train[1] : \n {}".format(text_train[1]))

type of text_train : <class 'numpy.ndarray'>
length of text_train : 25000
text_train[1] : 
 b'If you want Scream or anything like the big-studio horror product that we get forced on us these days don\'t bother. This well-written film kept me up thinking about all it had to say. Importance of myth in our lives to make it make sense, how children interpret the world (and the violence in it), our ransacking of the environment and ignorance of its history and legends.. all here, but not flatly on the surface. You could technically call it a "monster movie" even though the Wendigo does not take physical form until the end, and then it\'s even up to you and your beliefs as to what\'s happening with the legendary spirit/beast. Some standard thriller elements for those looking just for the basics and the film never bores, though in fact the less you see of the creature, the better. Fessenden successfully continues George Romero\'s tradition of using the genre as parable and as a discussion for

你可以看到，text_train是一个长度为75000的列表其申斑个元素是包含一条评论的字符串。我们打印出索引编号为1的评论。你还可以看到，评论中包含一些HTML换行符(< br  /> ) ,虽然这些符号不太可能对机器学习模型产生很大影响，但最好在继续下一步之前清洗数据并删除这种格式：

In [28]:
text_train = [doc.replace(b'<br />',b' ') for doc in text_train]

收集数据集时保持正类和反类的平衡，这样所有正面字符串和负面字符串的数量相等：

In [4]:
import numpy as np
# numpy.bincount() 方法计算每个元素的出现次数。每个 bin 值都是其索引的出现。还可以相应地设置 bin 大小。
print("Samples per class(training): {}".format(np.bincount(y_train)))

Samples per class(training): [12500 12500 50000]


用同样的方式加载测试数据集：

In [32]:
reviews_test = load_files("E:/clone/machine-learning/data/aclImdb/test/")
data_array = np.array(reviews_test.data)
target_array = np.array(reviews_test.target)
text_test, y_test = data_array, target_array
print("type of text_test : {}" .format(type(text_test))) 
print("Number of documents in test data: {}" .format(len(text_test))) 
print("Samples per class (test): {}".format(np.bincount(y_test))) 
text_test = [doc.replace(b'<br />',b' ') for doc in text_test]

type of text_test : <class 'numpy.ndarray'>
Number of documents in test data: 25000
Samples per class (test): [12500 12500]


In [18]:
from collections import Counter
element_counts = Counter(y_test)

# Print the result
for element, count in element_counts.items():
    print(f"Element {element} appears {count}")

Element 1 appears 12500
Element 0 appears 12500


我们要解决的住务如下：给定一条评论，我们希望根据修解论的文本内容对其分配一个 "正面 的 "或 “负面的"标签。这是一项标准的二分类任务。但是，文本数据并不是机器学习模型"以处理的格式。我们需要将文本的字符串表示转换为数值表示，从而可以对其应用机器学习算法。